# 💬 Week 7-8 · Notebook 02 · The Language of AI: Mastering Message Structures

**Module:** LangChain, Agents, & Advanced RAG  
**Project:** Manufacturing Copilot - Building Conversational Fluency

---

### Beyond Single Questions: The Structure of Dialogue

In the last notebook, we built a simple, single-shot chain. But real-world applications are rarely that simple. They involve conversations—a back-and-forth exchange between a user and the AI.

To manage these conversations, LangChain uses a critical abstraction: **Messages**.

Messages are specialized data structures that represent the different roles in a dialogue. They provide a standardized way to instruct the AI, capture user input, and record the AI's responses. Understanding these message types is the key to building everything from simple chatbots to complex, multi-turn agents.

In this notebook, we will explore the primary message types and learn how to use them to create a robust, stateful, and safe conversational experience for our Manufacturing Copilot. We'll simulate a real-world scenario where technicians in different locations, speaking different languages, collaborate to solve a problem, all orchestrated through a carefully managed message history.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to expertly manage conversational data in LangChain. You will learn to:

1.  **Differentiate Core Message Types:** Understand the specific roles of `SystemMessage`, `HumanMessage`, and `AIMessage` in structuring a conversation.
2.  **Construct a Chat History:** Build a list of message objects to represent a multi-turn dialogue.
3.  **Implement a Basic Conversational Chain:** Create a chain that uses message history to provide context-aware responses.
4.  **Simulate a Bilingual Dialogue:** Model a real-world scenario where the AI must handle and respond in multiple languages based on the conversation history.

## ⚙️ Step 1: Environment Setup

As before, we'll start by ensuring our environment is set up with the necessary libraries and API keys. We'll be using `langchain`, `langchain-openai`, and `python-dotenv`.

In [ ]:
# Install necessary packages
!pip install -q langchain langchain-openai python-dotenv

import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
    print("API Key not found. Please set it as an environment variable or replace 'YOUR_API_KEY'.")
else:
    print("OpenAI API Key loaded successfully.")

## ? Step 2: Understanding the Three Main Message Types

LangChain models conversations using a list of `Message` objects. Each message has a `content` (the text) and a `role`. There are three primary roles you will use constantly:

1.  **`SystemMessage`**:
    -   **Role:** The director or rule-setter for the AI.
    -   **Purpose:** It sets the context, personality, and constraints for the AI model. It tells the AI *how* to behave. A good system message is critical for guiding the AI's responses. It's usually the very first message in a conversation.
    -   **Example:** *"You are a helpful manufacturing assistant. Always respond concisely and include safety warnings."*

2.  **`HumanMessage`**:
    -   **Role:** The user or human in the conversation.
    -   **Purpose:** It represents the input from the person interacting with the AI.
    -   **Example:** *"The main conveyor belt has stopped. What are the first three things I should check?"*

3.  **`AIMessage`**:
    -   **Role:** The AI's response.
    -   **Purpose:** It represents the output generated by the language model. By including past AI messages in the history, you allow the model to remember what it has already said, which is crucial for coherent, multi-turn conversations.
    -   **Example:** *"1. Check for physical obstructions. 2. Verify the emergency stop is disengaged. 3. Check the motor's thermal overload switch. Always perform lockout-tagout before any physical inspection."*

Let's create a sample conversation history that simulates a real-world interaction at a manufacturing plant.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# --- 1. Define the Message History ---
# This list represents a conversation that has already happened.
conversation_history = [
    # The SystemMessage sets the overall tone and rules for the AI.
    SystemMessage(content="You are a bilingual manufacturing assistant. Your primary goal is to help technicians solve problems by referencing SOPs. Always prioritize safety and be concise. If the user speaks in Spanish, you MUST respond in Spanish."),
    
    # The first user query.
    HumanMessage(content="Press #7 is showing an overheating warning after we swapped the lubricant. What are the first things to check?"),
    
    # The AI's first response.
    AIMessage(content="Understood. For an overheating issue on Press #7 after a lubricant change, please perform the following initial checks:\n1. **Verify Lubricant Type:** Confirm that the new lubricant matches the specifications in SOP-122, Section 3.1.\n2. **Check Coolant Flow:** Ensure the coolant lines are not blocked and the pump is operational.\n\n**Safety Warning:** Perform a full lockout-tagout on Press #7 before any physical inspection."),
]

# --- 2. Add a New User Query ---
# Now, a new question comes in. This simulates a follow-up question from a different technician.
# This technician is in Monterrey, so they ask their question in Spanish.
new_human_message_spanish = HumanMessage(content="Gracias. Los sensores de temperatura del husillo principal marcan 95°C. ¿Hay algunos pasos adicionales que debamos seguir?")

# We add the new message to our history.
full_conversation = conversation_history + [new_human_message_spanish]


# --- 3. Create and Invoke the Conversational Chain ---
# We can now pass this entire conversation history to the model.
# The model will use the full context (including the system prompt and the previous turns) to generate a relevant and language-appropriate response.

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.3)

print("--- Sending Full Conversation to LLM ---")
for msg in full_conversation:
    print(f"[{msg.type.upper()}]: {msg.content}")

print("\n--- Generating AI Response ---")
# The `invoke` method can take a list of messages directly.
ai_response = llm.invoke(full_conversation)

print(f"\n[AI]: {ai_response.content}")

### Interpretation of the Result

As you can see, the AI's response is in Spanish, and it directly addresses the new question about the spindle temperature. This demonstrates the power of message history:

1.  **Context-Awareness:** The AI knew the conversation was about "Press #7" and an "overheating issue" because of the earlier messages in the history.
2.  **Instruction Following:** The AI responded in Spanish because the `SystemMessage` instructed it to, and the last `HumanMessage` was in Spanish.
3.  **Stateful Dialogue:** By maintaining and passing the list of messages, we have created a stateful conversation where the AI can remember what has been discussed previously.

This simple list of messages is the fundamental data structure you will use to build all conversational applications in LangChain, from simple chatbots to complex agents.

## ✅ Congratulations and Next Steps!

You have now mastered the fundamental data structure for all conversational AI in LangChain: the **message list**.

You've learned how to use `SystemMessage`, `HumanMessage`, and `AIMessage` to create a structured, stateful conversation history. You also saw how the model uses this entire history to provide context-aware and instruction-following responses, even switching languages on the fly.

### Key Takeaways:
-   **Messages are the API for Chat Models:** A list of messages is the universal format for interacting with modern chat-based LLMs.
-   **The `SystemMessage` is Your Control Panel:** It's the most important tool you have for guiding the AI's personality, tone, and behavior.
-   **History Provides Context:** The sequence of `HumanMessage` and `AIMessage` objects gives the AI its memory, allowing for coherent, multi-turn dialogues.

In the next notebook, **`03_prompt_templates_chains.ipynb`**, we will combine our knowledge of messages with prompt templates to create more dynamic and reusable conversational chains, further enhancing the capabilities of our Manufacturing Copilot. Let's proceed!